# artists.ipynb
get alphabetized list of artists appearing on a specified playlist
<br>
## Todo
- find a way to append new artists to correct category and label as new

In [ ]:
# setup 
import spotipy
import spotipy.util as util
import os
import sys
import pprint
import string

## file setup

In [ ]:
## ask for basename - make sure the file is in the working directory of this script
basename = input("name of artist file: ")

## find file
path = os.getcwd() + "/" + basename + ".txt"

## do checks
if os.path.exists(path) and os.path.isfile(path):
    # add each artist to set
    artists = set([])
    # open
    with open(path) as fp:  
        line = fp.readline()
        while line:
            # filter out headers and empty lines
            if line.strip()[:3] != '---' and line.strip()[:3] != '':
                artists.add(line.strip())
            line = fp.readline()
else:
    # shit happened
    print("bad file: '%s'" % path)

In [ ]:
## create dictionary
# set keys as all letters and '#'(other)
# set values as empty lists
artist_dict = dict.fromkeys(string.ascii_uppercase, [])
artist_dict['#'] = []

for artist in artists:
    first_letter = artist[:1].upper()
    # first letter is not a letter, so bunch into misc. '#' key
    if(first_letter not in artist_dict.keys()):
        artist_dict['#'].append(artist)
    else:
        if artist_dict[first_letter] == []:
            artist_dict[first_letter] = [artist]
        else:
            artist_dict[first_letter].append(artist)

# sort and write to file
with open('n_p-artists.txt', 'w') as fhandle:
    for key in artist_dict.keys():
        fhandle.write("--- " + key + " ---\n")
        # write artists
        for artist in artist_dict[key] :
            fhandle.write(artist + "\n")
        fhandle.write("\n\n")

In [ ]:
## API info
# get this info here: https://developer.spotify.com/dashboard/applications
SPOTIPY_CLIENT_ID='client-id'
SPOTIPY_CLIENT_SECRET='client-secret'
SPOTIPY_REDIRECT_URI='redirect-url'

# set environment variables
# this may or may not be necessary
os.environ['SPOTIPY_CLIENT_ID']=SPOTIPY_CLIENT_ID
os.environ['SPOTIPY_CLIENT_SECRET']=SPOTIPY_CLIENT_SECRET
os.environ['SPOTIPY_REDIRECT_URI']=SPOTIPY_REDIRECT_URI
my_username = "my-username"

# set permissions scope
my_scope = 'user-library-read playlist-modify-private playlist-modify-public user-read-playback-state'

# set up request token
token = util.prompt_for_user_token(
        username=my_username,
        scope=my_scope,
        client_id=SPOTIPY_CLIENT_ID,
        client_secret=SPOTIPY_CLIENT_SECRET,
        redirect_uri=SPOTIPY_REDIRECT_URI)

In [ ]:
def get_playlist_id(playlist, playlists):
    '''
    given a list of dictionaries of playlist attributes, check if user specified playlist exists
    
    Arguments:
        playlist - string, name of playlist to search for
        playlists - list of possible playlists
    
    Returns:
        int     - index in list of playlists where desired playlist found. -1 if not found 
    '''
    for i in range(0, len(playlists)):
        if playlists[i]['name'] == playlist:
            return playlists[i]['id']
    return None

## get name of desired playlist
playlist = ''
playlist_id = None
# 
index = -1
tracks = []

if token: 
    sp = spotipy.Spotify(auth=token)
    sp.trace = False
    
    ## get tracks of that playlist
    # function requires user id, not username, and playlist id, not name, so get ids
    # user id
    user_id = (sp.me())['id']
    
    ## find playlist, then get its id
    # results are paginated so interate through them
    all_playlists = []
    results = sp.current_user_playlists()
    all_playlists = results['items']
    
    while results['next']:
        results = sp.next(results)
        all_playlists.extend(results['items'])
    
    ## check if playlist exists, reprompt so long as it doesn't
    while playlist_id is None:
        playlist = input("playlist to retrieve artists from: ")
        playlist_id = get_playlist_id(playlist, all_playlists)

    # don't really need try catch, since this function is not called until the playlist id is not None,
    # but let's be extra safe and user friendly
    try:
        ## get tracks
        tracks = sp.user_playlist_tracks(user_id, playlist_id)
        ## get artist names
        artists = set([])
        for i in range(0, len(tracks['items'])):
            for j in range(0, len(tracks['items'][i]['track']['artists'])):
                name = tracks['items'][i]['track']['artists'][j]['name']
                # getting weird occurence of an empty artist name, so filter out
                if name != '':
                    # is a set, so only new artists will be added
                    artists.add(name)

        ## create and fill dictionary
        # set keys as all letters and '#'(other)
        # set values as empty lists
        artist_dict = dict.fromkeys(string.ascii_uppercase, [])
        artist_dict['#'] = []

        # fill
        for artist in artists:
            first_letter = artist[:1].upper()
            if first_letter != '':
                if(first_letter not in artist_dict.keys()):
                    artist_dict['#'].append(artist)
                else:
                    if artist_dict[first_letter] == []:
                        artist_dict[first_letter] = [artist]
                    else:
                        artist_dict[first_letter].append(artist)

        ## sort and write to file
        out_file = playlist + '-artists.txt'
        with open(out_file, 'w') as fhandle:
            for key in artist_dict.keys():
                fhandle.write("--- " + key + " ---\n")
                # write artists
                for artist in artist_dict[key] :
                    fhandle.write(artist + "\n")
                fhandle.write("\n\n")

    except Exception as e:
        print(e);
else:
    print("Can't get token for: %s" % username)
    raise SystemExit